# Creation
Algorithm: Given an operator $\rho_{i,j}\left|i\right>\left<j\right|$ with indices $i,j\in\mathbb Z_d^n$:
(<mark>todo: update max bond dimension to shrink after middle index reached</mark>)
1. Reshape: $\rho_{i,j}= P_{(i_1\cdots i_n),(j_1\cdots j_n)}$ has dimensions $d^n\times d^n$
2. Permute: $P_{(i_1\cdots i_n),(j_1\cdots j_n)}= T_{(i_1j_1),\cdots ,(i_nj_n)}$ has dimensions $d^2\times\cdots \times d^2$ ($n$ times)
3. Collect: $T_{(i_1j_1),(i_2j_2\cdots i_nj_n)}$ has dimensions $d^2\times d^{2(n-1)}$
4. First site:
	- SVD: $T_{(i_1j_1),(i_2j_2\cdots i_nj_n)}=\sum_{\alpha\in \mathbb Z_{D_1}} U_{(i_1j_1),\alpha}S_{\alpha,\alpha} (V^\dagger)_{\alpha,(i_2j_2\cdots i_nj_n)}$, where $D_1\in \{1,\cdots,d^2\}$ is determined by truncation 
	- Save: $M^{(1)}_{\alpha,u,\beta}=\delta_{\alpha,1}U_{u,\beta}$ has dimensions $1\times d^2\times D_1$
	- Update: $T^{(1)}_{(\alpha i_2j_2),(i_3j_3\cdots i_nj_n)}=S_{\alpha,\alpha}(V^\dagger)_{\alpha,(i_2j_2\cdots i_nj_n)}$ has dimensions $D_1 d^2\times d^{2(n-2)}$
5. Inner sites: For $k\in \{2,\cdots,n-1\}$:
	- SVD: $T^{(k)}_{(\alpha i_kj_k),(i_{k+1}j_{k+1}\cdots i_nj_n)}=\sum_{\beta\in \mathbb Z_{D_k}} U_{(\alpha i_kj_k),\beta}S_{\beta,\beta} (V^\dagger)_{\beta,(i_{k+1}j_{k+1}\cdots i_nj_n)}$, where $D_k\in\{1,\cdots,D_{k-1}d^2\}$
	- Save: $M^{(k)}_{\alpha,u,\beta}=U_{(\alpha u),\beta}$ has dimensions $D_{k-1}\times d^2\times D_k$
	- Update: $T_{(\alpha i_{k+1}j_{k+1}),(i_{k+2}j_{k+2}\cdots i_nj_n)}=S_{\alpha,\alpha} (V^\dagger)_{\alpha,(i_{k+1}j_{k+1}\cdots i_nj_n)}$ has dimensions $D_k d^2\times d^{2(n-k-1)}$
6. Last site:
	- Save: $M^{(n)}_{\alpha,u,\beta}=T^{(n-1)}_{(\alpha,u),\beta}$ has dimensions $D_{n-1}\times d^2\times 1$ (note that $T^{(n-1)}_{(\alpha i_nj_n),\beta}$ has dimensions $D_{n-1} d^2\times 1$)
7. Return $\{M^{(k)}\}_{k\in\mathbb Z_n}$

In [1]:
import numpy as np
from numpy import linalg as la


# Given a density matrix, return its matrix product operator representation
# Indices are stored in the following format: (bond,phys,bond)
def state_to_mpo(state, n, d, verbose=False, **kwargs):
    # If not given a maximum bond dimension, set it to the maximum possible - d^(4n) (TODO)?
    #TODO: add a truncation magnitude cuttoff instead of just max_bd
    max_bd = kwargs.get('max_bd', d**(4*n))
    
    # Constructing environment tensor                                                                              
    tensor_shape = tuple([d]*(2*n))
    tensor = state.reshape(tensor_shape)
    tensor_axes = [i for i in range(2*n)]
    T_axes = [int(i/2) if i%2 == 0 else int(n + (i/2)) for i in range(2*n)]
    d2 = d**2
    T = np.moveaxis(tensor, tensor_axes, T_axes)
    T = T.reshape((d2,d2**(n-1)))
    if verbose:
        print(0, "Initial T:\t\t\t\t", T.shape)

    # First site
    mpo = [0]*n
    U, S, Vt = la.svd(T, full_matrices = False)
    U = U[:,:max_bd]
    S = S[:max_bd]
    Vt = Vt[:max_bd,:]
    mpo[0] = U.reshape((1,U.shape[0],U.shape[1]))
    T = np.dot(np.diag(S),Vt).reshape(S.size*d2,int(Vt.shape[1]/d2))
    if verbose:
        print(1, U.shape, Vt.shape, "\t->", mpo[0].shape, "\t", T.shape, "\t", S.size)

    # Interior sites
    for i in range(1,n-1):
            U, S, Vt = la.svd(T, full_matrices = False)
            U = U[:,:max_bd]
            S = S[:max_bd]
            Vt = Vt[:max_bd,:]
            mpo[i] = U.reshape((int(U.shape[0]/d2),d2,U.shape[1]))
            T = np.dot(np.diag(S),Vt).reshape((S.size*d2,int(Vt.shape[1]/d2)))
            if verbose:
                print(i+1, U.shape, Vt.shape, "\t->", mpo[i].shape, "\t", T.shape, "\t", S.size)

    # Last site
    mpo[n-1] = np.dot(np.diag(S),Vt).reshape((S.size, Vt.shape[0], 1))
    if verbose:
        print(n, "Final Matrix:\t\t  ", mpo[n-1].shape)

    return mpo

# Frobenius inner product tr[A^\dagger B] between two states A and B represented by mpos 
# norm: Frobenius normalization of local basis elements used in MPO representation (norm = Tr[A_u^\dagger A_v])
# Assumptions:
#       len(m1) = len(m2)                                       [same length]
#       m1[i].shape = m2[i].shape = (bond,phys,bond)            [same bond and phys dim at each site]
#       m1[0].shape = (1,_,_) and m1[-1].shape = (_,_,1)        [closed boundary conditions]
def inner_prod(m1, m2, norm=1):
        # Extract length, phys dim, and first site bond dim
        n = len(m1)

        #print(m1[0][0].shape, m2[0][0].shape)
        # First site
        M = np.tensordot(m1[0][0].conj(),m2[0][0],axes=([0,0]))      # sum over physical indices (grab the first and only matrix due to fixed bdry cond.)

        #print(0,m1[0].shape,M.shape)
        # Rest of contraction
        for i in range(1,n):
                M = np.tensordot(M,m1[i].conj(),axes=([0,0]))
                M = np.tensordot(M,m2[i],axes=([0,1],[0,1]))
                #print(i,m1[i].shape, M.shape)
        return M[0][0]*norm

We can run this algorithm on a randomly generated density matrix:

In [2]:
# Generate a mixed state of randomly sampled pure qudit states                                                 
# d - onsite hilbert space dimension
# N - number of sites
# M - number of pure states used
def random_mixed_state(N,d=3,M=10):
        states = [random_state(N,d) for i in range(M)]
        weights = [np.random.uniform(low=0.0,high=1.0) for i in range(M)]
        total_weight = sum(weights)
        state = np.zeros((d**N,d**N))
        for i in range(M):
                state = state + (weights[i]/total_weight) * np.outer(states[i],states[i].conj())
        return state

# Generate a random qudit state
# d - onsite hilbert space dimension
# N - number of sites
def random_state(N,d=3):
        normals = [np.random.standard_normal(size=d**N) for i in range(2)]
        state = np.array([normals[0][i] + 1j * normals[1][i] for i in range(d**N)])
        state /= la.norm(state)
        return state

I should expect to see the following dimensions of my matrices and local tensor after performing each SVD for $d=3,n=5$: The density matrix has dimensions $3^5\times 3^5$ and the environment tensor $T$ starts out with dimensions $9\times 9^4$. After each step, we should expect $D_k=9^k$ <mark>(TODO: update this)</mark>

In [14]:
d = 3
n = 3
rho = random_mixed_state(n,d)
m_rho = state_to_mpo(rho,n,d,verbose=True)

True
0 Initial T:				 (9, 81)
1 (9, 9) (9, 81) 	-> (1, 9, 9) 	 (81, 9) 	 9
2 (81, 9) (9, 9) 	-> (9, 9, 9) 	 (81, 1) 	 9
3 Final Matrix:		   (9, 9, 1)


# Inner products
The Frobenius norm overlap between two operators is defined to be
$$\left<\rho|\sigma\right>=\mathrm{Tr}\left[\rho^\dagger\sigma\right]=\sum_u \left(\prod_i \rho^{(i)u_i}\right)^*\left(\prod_j \sigma^{(i)u_j}\right),$$
where $*$ denotes element-wise complex conjugation. This can be calculated in a linear number of computations in $d$ and $n$ and cubic in $D^\sigma$ and $D^\rho$ like so:
1. $M_{\alpha\beta} = \sum_{u\in \mathbb Z_d}(\rho^{(1)}_{1u\alpha})^*\sigma^{(1)}_{1u\beta}$
2. For $k\in\{2,\cdots,n\}:$
    - $M_{\alpha\beta}\leftarrow \sum_{u\in \mathbb Z_d,\mu\in\mathbb Z_{D^\rho_k},\mu\in\mathbb Z_{D^\sigma_k}}(\rho^{(k)}_{\mu u \alpha})^*M_{\mu \nu}\sigma^{(k)}_{\nu u \beta}$
3. Return $M_{11}$

Note that the matrix elements are stored like so: $\rho^{(k)u}_{\alpha\beta}=\rho^{(k)}_{\alpha u \beta}$, and $D^\rho_k$ denotes the bond dimension of $\rho^{(k)}$ at site $k$ 

In [4]:
print(inner_prod(m_rho,m_rho), np.trace(np.dot(rho.conj().T,rho)))

(0.1408766099270832-2.2768245622195593e-18j) (0.14087660992708334-1.8220653968812746e-20j)


## Moving to phase space

We formed our MPO like so: $\rho_{ij}\left|i\right>\left<j\right|=\rho_{(i_1\cdots i_n)(j_1\cdots j_n)}(\left|i_1\right>\otimes \cdots \otimes \left|i_n\right>)(\left<j_1\right|\otimes \cdots \otimes \left<j_n\right|)=\sum_{i_kj_k\alpha _k}\rho^{(1)}_{\alpha_0(i_1j_1)\alpha_1}\left|i_1\right>\left<j_1\right|\otimes\cdots\otimes \rho^{(n)}_{\alpha_{n-1}(i_nj_n)\alpha_n}\left|i_n\right>\left<j_n\right|$. This gave us a matrix product operator of the form $\sum_{u,\alpha} \otimes_k \rho^{(k)u_k}_{\alpha_k\alpha_{k+1}}C_{u_k}$, where $C_u=\left|u_1\right>\left<u_2\right|$. We want to represent it in phase space, i.e. we want to find the coefficents $\tidle \rho$ satisfying $\sum_{u,\alpha} \otimes_k \rho^{(k)u_k}_{\alpha_k\alpha_{k+1}} C_{u_k}=\sum_{u,\alpha} \otimes_k \tilde{\rho}^{(k)u_k}_{\alpha_k\alpha_{k+1}} A_{u_k}$. The following choice works:

$$\tilde{\rho}^{(k)u}_{\alpha\beta}=d^{-1}\sum_v\rho^{(k)v}_{\alpha\beta}\mathrm{Tr}\left[C_uA_v\right].$$

Note that $\mathrm{Tr}[C_u A_v]=\sum_k \left<k|u_1\right>\left<u_2|A_v |k\right>=\left<u_1|A_v|u_2\right>=(A_v)_{u_1,u_2}$, so the change-of-basis coefficients are proportional to the matrix elements of the phase space operators. Since $\mathrm{Tr}[A_uA_v]=d\delta_{u,v}$, the phase space basis is not normalized. This means that we pick up a Hilbert space factor when computing inner products:

$$\mathrm{Tr}[\rho^\dagger \sigma]=d^n\left<\rho|\sigma\right>_A,$$

where $\left<\rho|\sigma\right>_A$ denotes the contraction of the matrix product representations of the states $\rho$ and $\sigma$ in the phase space basis $A_u$.

In [5]:
import cmath
import math

d = 3   # qutrit

# qutrit X and Z matrices
omega = cmath.exp(cmath.pi*2j/d)
X = np.array([[0,0,1],[1,0,0],[0,1,0]])
Z = np.array([[1,0,0],[0,omega,0],[0,0,omega**2]])

# 1-qudit pauli matrices
T = np.array([[omega**(-2*a*b)*np.dot(la.matrix_power(Z,a),la.matrix_power(X,b)) for b in range(d)] for a in range(d)])

# 1-qudit phase space operators (TODO: change the sum below to a numpy sum)
A_0 = (1./d)*sum([sum([T[a,b] for b in range(d)]) for a in range(d)])
A = np.zeros((d,d,d,d), dtype=np.complex128)
for i in range(d):
    for j in range(d):
        A[i,j] = np.dot(T[i,j],np.dot(A_0,np.conj(T[i,j].T)))
A = A.reshape((d**2,d,d))

# Computational basis C_u = |u_1><u_2|
#comp_basis = np.zeros((d,d,d,d), dtype=np.complex128)
#for i in range(d):
#    for j in range(d):
#        comp_basis[i,j,i,j] = 1
#comp_basis = comp_basis.reshape((d**2,d,d))

# Change of basis coefficients Tr[A_u^\dagger C_u] = Tr[A_u C_u]
#ps_coefficients = np.zeros((d**2,d**2),dtype=np.complex128)
#for i in range(d**2):
#    for j in range(d**2):
#        ps_coefficients[i,j] = 1./d*np.trace(np.dot(A[i],comp_basis[j]))
ps_coefficients = 1./d*A.reshape((d**2,d**2))

# change of basis coefficients
# TODO: change this to a numpy array
#ps_coefficients = [[np.trace(np.dot(A[u],comp_basis[v])) for u in range(d**2)] for v in range(d**2)]

#TODO: get rid of for loops
def basis_change(mpo, coefficients):
    n = len(mpo)
    _,d,_ = mpo[0].shape
    
    res = [None]*n
    for k in range(n):
        res[k] = np.zeros(mpo[k].shape, dtype=np.complex128)
        temp = np.tensordot(mpo[k],coefficients,axes=([1,1])) # sum over physical indices
        temp = np.moveaxis(temp, [0,1,2], [0,2,1])
        res[k] = temp
    return res

Some checks to make sure that we did the change of basis correctly:

In [6]:
# Check properties of basis elements
for i in range(d**2):
    for j in range(d**2):
        if abs(np.trace(np.dot(A[i],A[j])).imag) > 1e-14:
            print(i,j,"complex trace error!")
        if not(np.allclose(A[i].conj().T,A[i])):
            print(i,j,"hermiticity error!")
        if i == j:
            if abs(np.trace(np.dot(A[i],A[j])).real - d) > 1e-14:
                print(i,j,"normalization error!")
        else:
            if abs(np.trace(np.dot(A[i],A[j])).real) > 1e-14:
                print(i,j,"normalization error!")

# Check that inner product in new basis agrees with old basis
m_rho_ps = basis_change(m_rho, ps_coefficients)
if (abs(inner_prod(m_rho_ps,m_rho_ps,norm=d**n).imag) > 1e-12):
    print("Inner product in new basis is imaginary")
if (abs(inner_prod(m_rho_ps,m_rho_ps,norm=d**n).real - inner_prod(m_rho,m_rho).real) > 1e-12):
    print("Inner product in new basis does not match")

We can now calculate the Wigner function and estimations of magic:

In [7]:
# n-dimensional phase space operators:
A_n = np.zeros((d**(2*n),d**n,d**n),dtype=np.complex128)
tensor_shape = tuple([d**2]*n)
for u in range(d**(2*n)):
    index = np.unravel_index(u,tensor_shape)
    temp = np.identity(1,dtype=np.complex128)
    for i in range(n):
        temp = np.kron(temp, A[index[i]])
    A_n[u] = temp
    
# Wigner function
def wigner(state, n, d):
    W = np.zeros((d**(2*n)))
    M = 0. #mana
    N = 0. #new quantifier
    for u in range(d**(2*n)):
        W[u] = 1./d*np.trace(np.dot(state,A_n[u]))
        if W[u].real < 0:
            M -= W[u].real
            N += (W[u].real)**2
    N = math.sqrt(N)
    return W, M, N

We now generate a random hermitian operator (not necessarily normalized) $\sigma$ whose matrix product operator representation in the phase space basis has only non-negative elements. We then vary $\sigma$ to reduce the squared distance

$$d(\rho,\sigma)^2=\mathrm{Tr}[(\rho-\sigma)^\dagger(\rho-\sigma)]=\mathrm{Tr}[\rho^\dagger\rho]-\mathrm{Tr}[\rho^\dagger\sigma]-\mathrm{Tr}[\sigma^\dagger\rho]+\mathrm{Tr}[\sigma^\dagger\sigma],$$

in the space of non-negative matrix product operators (i.e. constrained optimization with the constraints $\sigma^{(k)}_{\alpha,u,\beta}\geq 0$). Note that $\sigma$ is a hermitian operator because it has a real Wigner function: 
$$\sigma^{(k)}_{\alpha,u,\beta}\geq 0\Rightarrow\left(\sum_\alpha \prod_k \sigma^{(k)}_{\alpha_k,u_k,\alpha_{k+1}}\right)\geq 0\ \forall  u\Rightarrow \mathrm{Tr}[\sigma A_{u}]\in\mathbb R\ \forall u$$
Since the phase space operators are hermitian and complete, this implies that
$$\sigma^\dagger=\left(d^{-n}\sum_u \mathrm{Tr}[\sigma A_u]A_u\right)^\dagger=d^{-n}\sum_u \mathrm{Tr}[\sigma A_u]^*A_u^\dagger=d^{-n}\sum_u \mathrm{Tr}[\sigma A_u]A_u=\sigma.$$
Since trace is cyclic, the squared distance simplifies to

$$d(\rho,\sigma)^2=\mathrm{Tr}[\rho^\dagger\rho]-2\mathrm{Tr}[\sigma^\dagger\rho]+\mathrm{Tr}[\sigma^\dagger\sigma]$$

In [8]:
import scipy as sp
from scipy.optimize import minimize

#TODO: seed 

def dist_sq(A, B, norm):
    return inner_prod(A,A,norm) - 2*inner_prod(A,B,norm) + inner_prod(B,B,norm)

def dist(A, B, norm):
    return math.pow(dist_sq(A,B,norm),1./2)

def cost_fct(i, x, A, B, norm):
    A[i] = x.reshape(A[i].shape)
    return inner_prod(A, B, norm)

# Generate random MPO with fixed bond dimension and random positive coefficients
def gen_mpo(n, d, bd):
    mpo = [None] * n
    mpo[0] = np.random.rand(1,d,bd)
    for i in range(1,n-1):
        mpo[i] = np.random.rand(bd,d,bd)
    mpo[n-1] = np.random.rand(bd,d,1)
    return mpo
    
def optimize(A, B, nswp = 20, opt_method = 'L-BFGS-B'):
    # The true distance between B and the positive-repr states is N:
    _, _, true_dist = wigner(B)
    print("Initial distance: %g, True distance: %g" % (dist(A,B), true_dist))

    # Optimize D(A||B) on A to find closest positive repr MPO to B
    print("Optimization method: %s" % opt_method)
    print("Sweep Site Distance Dist-True_Dist")
    #print("Half-sweep D(A||B) D(A||B)-D(A*||B) E(A) H_2(A)\n")
    sys.stdout.flush()

    n = len(A)
    _,d,_ = A.shape
    dn = d**n
    b = [(0, None) for i1 in range(D_A*(d**2))] #todo: need to impose constraints dependent on bond dim at that site
    start_time = time.time()
    for i in range(nswp):
        for j in range(n):
            A[j] = minimize(lambda x:cost_fct(0,x,A,B,dn), A[j], method=opt_method, bounds=b).x
            print("%d %d %g %g %g %g" % (i, j, dist(A,B,dn), dist(A,B,dn)-true_dist)) #vn_entropy(A), r_entropy(A,2)))
            sys.stdout.flush()

    print("\nTime elapsed during optimization: %g" % (time.time() - start_time))

Unfortunately, the squared distance comes out to be complex, which isn't good.. let's try to debug:

In [12]:
A = gen_mpo(n,d**2,10)
norm = d**n
B = m_rho_ps
print(inner_prod(A,A,norm))
print(inner_prod(B,B,norm))
print(inner_prod(A,B,norm))
print(inner_prod(B,A,norm))
print(np.amax( [np.amax(np.abs(np.imag(B[j]) ) ) for j in range(n) ] ))

# Check that an mpo in the phase space basis represents a hermitian operator
def check_real_coefficients(mpo):
    tensor_shape = tuple([d**2]*n)
    for u in range(d**(2*n)):
        index = np.unravel_index(u,tensor_shape)
        coefficient = np.identity(1,dtype=np.complex128)
        for i in range(n):
            coefficient = np.dot(coefficient, mpo[i][:,index[i],:])
        if (coefficient[0][0].imag > 1e-16):
            print("Coefficient " + repr(index) + " is complex with value " + repr(coefficient[0][0]))

check_real_coefficients(A)
check_real_coefficients(B)


3163211.9975912566
(0.1408766099270832-1.463672932855431e-18j)
(69.76761528289639+3.612946370688718j)
(69.76761528289639-3.6129463706887197j)
0.29609164130372767
Coefficient (0, 0, 3) is complex with value (0.0005861330502266037+0.0010320546977816113j)
Coefficient (0, 0, 4) is complex with value (0.0022732585829827153+0.0005776751315375542j)
Coefficient (0, 0, 6) is complex with value (0.00017637953615237266+0.0005165055485560994j)
Coefficient (0, 0, 8) is complex with value (0.0021682462672570476+0.001207811439336411j)
Coefficient (0, 1, 1) is complex with value (0.0002349332475655643+0.0056954178859802964j)
Coefficient (0, 1, 2) is complex with value (0.004089904549234537+0.0008071546737729413j)
Coefficient (0, 1, 3) is complex with value (-0.0027633240479209217+0.001621553222590477j)
Coefficient (0, 2, 2) is complex with value (-0.00018109926064481264+0.0008712439062930504j)
Coefficient (0, 2, 3) is complex with value (0.0001247253938123248+0.0005796820932602038j)
Coefficient (0, 2,